In [1]:
import pandas as pd
import math
from collections import namedtuple, defaultdict

In [2]:
from ortools.linear_solver import pywraplp
import asyncio

In [3]:
Point = namedtuple("Point", ['x', 'y'])
Facility = namedtuple("Facility", ['index', 'setup_cost', 'capacity', 'location'])
Customer = namedtuple("Customer", ['index', 'demand', 'location'])

In [4]:
def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

In [8]:
def solve_it(input_data, required_solution, use_memory=True):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    parts = lines[0].split()
    facility_count = int(parts[0])
    customer_count = int(parts[1])
    
    facilities = []
    for i in range(1, facility_count+1):
        parts = lines[i].split()
        facilities.append(Facility(i-1, float(parts[0]), int(parts[1]), Point(float(parts[2]), float(parts[3])) ))

    customers = []
    for i in range(facility_count+1, facility_count+1+customer_count):
        parts = lines[i].split()
        customers.append(Customer(i-1-facility_count, int(parts[0]), Point(float(parts[1]), float(parts[2]))))

    # build a trivial solution
    # Define the problem parameters
    distances = defaultdict(dict)
    if use_memory:
        for i in range(facility_count):
            for j in range(customer_count):
                distances[i][j] = length(facilities[i].location, customers[j].location)

    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Define the decision variables
    facilities_var = [solver.IntVar(0, 1, f'facility_{i}') for i in range(facility_count)]
    customers_var = [[solver.IntVar(0, 1, f'customer_{j}{i}') for i in range(customer_count)] for j in range(facility_count)]
    
    # Define the constraints
    for i in range(customer_count):
        solver.Add(sum(customers_var[j][i] for j in range(facility_count)) == 1)
        
    for i in range(facility_count):
        solver.Add(sum(customers_var[i][j] * customers[j].demand for j in range(customer_count)) <= facilities[i].capacity)

    for i in range(facility_count):
        for j in range(customer_count):
            solver.Add(customers_var[i][j] <= facilities_var[i])
        
    # Define the objective function
    fixed_cost = sum(facilities_var[i] * facilities[i].setup_cost for i in range(facility_count))
    if use_memory:
        transport_cost = sum(distances[i][j] * customers_var[i][j] for i in range(facility_count) for j in range(customer_count))
    else:
        transport_cost = sum(length(facilities[i].location, customers[j].location) * customers_var[i][j] for i in range(facility_count) for j in range(customer_count))
    objective = solver.Minimize(fixed_cost + transport_cost)    
    
    # Solve the problem
    
    solver.SetNumThreads(6)
    solver.SetTimeLimit(240 * 1000)
    solver.Solve()
    
    print(solver.Objective().Value())
    # Print the solution
    solution = []
    for j in range(customer_count):
        for i in range(facility_count):
            if customers_var[i][j].solution_value():
                solution.append(i)
    
    return solution

In [9]:
def get_result(file_name, reqired_solution, use_memory=True):
    with open('data/'+file_name, 'r') as input_data_file:
        input_data = input_data_file.read()
    solution = solve_it(input_data, reqired_solution, use_memory)
    print(solution)
    res = pd.DataFrame(solution, columns=['Order'])
    res.to_excel(file_name+'_res.xlsx', index=False, header=True)

In [10]:
get_result('1_fl_25_2', 3270000)

3269821.320530881
[7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 10, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 21, 7, 7, 7, 7, 7, 7, 16, 7, 7, 11, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


In [202]:
get_result('2_fl_50_6', 3733000)

SOLUTION # 1 HAS OBJ 3732793.43377155
[28, 24, 19, 25, 14, 15, 34, 3, 9, 24, 35, 42, 41, 24, 49, 3, 16, 26, 43, 45, 45, 41, 9, 34, 9, 13, 19, 8, 38, 24, 24, 7, 9, 25, 31, 33, 28, 9, 28, 25, 38, 40, 35, 7, 2, 19, 40, 25, 41, 9, 34, 44, 41, 18, 35, 5, 9, 31, 14, 35, 31, 35, 44, 43, 9, 4, 8, 14, 25, 45, 28, 33, 41, 39, 42, 6, 8, 35, 6, 24, 40, 47, 31, 24, 31, 24, 24, 45, 34, 9, 7, 2, 5, 39, 25, 35, 24, 40, 31, 3, 47, 6, 39, 16, 31, 44, 2, 16, 9, 13, 8, 9, 47, 35, 15, 24, 43, 25, 42, 16, 35, 28, 34, 35, 13, 5, 8, 35, 18, 11, 38, 39, 43, 41, 47, 44, 9, 41, 9, 38, 38, 28, 19, 9, 28, 28, 42, 41, 47, 9, 35, 38, 29, 8, 45, 49, 16, 25, 26, 31, 38, 5, 49, 10, 7, 40, 44, 29, 34, 10, 2, 41, 13, 31, 40, 28, 35, 49, 44, 33, 4, 2, 16, 47, 28, 9, 3, 31, 11, 16, 31, 25, 5, 42, 13, 31, 8, 40, 44, 45]


In [26]:
get_result('3_fl_100_7', 1970)

[70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70]


In [23]:
get_result('1_fl_25_2', 3270000, False)

[7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 10, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 21, 7, 7, 7, 7, 7, 7, 16, 7, 7, 11, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


In [30]:
get_result('4_fl_200_7', 4712000, False)

[]


In [ ]:
get_result('5_fl_500_7', 27007000, False)